In [12]:
import pandas as pd
from omegaconf import OmegaConf
from lib.uncertainty import Simulate
from prettytable import PrettyTable

In [13]:
EXPERIMENTS = [
    '2026-01-20_11-24', # Dense
    '2026-01-20_12-06', # Similar
    '2026-01-20_12-23', # Oracle
    '2026-01-20_12-34' # Random
]

strategy = {
    'oracle': 'Oracle',
    'random': 'Random',
    'similar': 'Similar',
    'dense': 'Dense'
}

In [14]:
results_all = {}

for experiment in EXPERIMENTS:
    cfg = OmegaConf.load(f'../results/{experiment}/config.yaml')
    results = pd.read_json(f'../results/{experiment}/results.json')
    results['correct_query'] = True
    results['retriever_strategy'] = cfg.retriever.strategy

    results_all[cfg.retriever.strategy] = results

In [15]:
simulate = Simulate(cfg)

success_rates = {}
for experiment, results in results_all.items():
    success_rates[experiment] = simulate.compute_uncertainty(results)

In [16]:
t = PrettyTable(field_names=['Retriever Strategy', 'Query', 'Retreiver', 'Generator', '# Tests'])

for experiment, success_rate in success_rates.items():
    t.add_row([
        strategy[experiment],
        f"{success_rate['q']['mean']:.2%} ± {success_rate['q']['std']:.4%}",
        f"{success_rate['r']['mean']:.2%} ± {success_rate['r']['std']:.4%}",
        f"{success_rate['g']['mean']:.2%} ± {success_rate['g']['std']:.4%}",
        len(results_all[experiment])
    ])

t

Retriever Strategy,Query,Retreiver,Generator,# Tests
Dense,99.99% ± 0.0099%,65.59% ± 0.4775%,78.61% ± 0.4093%,10000
Similar,99.99% ± 0.0099%,0.01% ± 0.0118%,58.60% ± 0.4856%,10000
Oracle,99.99% ± 0.0099%,99.98% ± 0.0119%,91.34% ± 0.2814%,10000
Random,99.99% ± 0.0099%,3.41% ± 0.1829%,4.31% ± 0.1995%,10000


In [17]:
cond = simulate.load_conditionals(results_all['dense'])

cond['r']['b']['q1'].mean

0.6559688062387522